In [ ]:
# the first part of this script collects the homepages from the internet archive and save them in a .txt

import urllib.request
import json, codecs,time
from bs4 import BeautifulSoup
from collections import Counter


# the name of the website as it appears in the URL, for example nytimes, guardian, lastampa
website_name = "de.sputniknews"

# the precise homepage, from which you want to collect news (this step is needed as sometimes it could point to a subdomain, e.g. theguardian.com/uk)
url_website = "de.sputniknews.com"

year = "2014"
year_to = "2018"

In [ ]:



url = "http://web.archive.org/cdx/search/cdx?url="+url_website+"&matchType=url&&collapse=timestamp:8&limit=15000&filter=!mimetype:image/gif&filter=!mimetype:image/jpeg&from="+ year + "01&to="+ year_to +"12&output=json&limit=1"

http = urllib.request.urlopen(url)
soup = http.read()                            
http.close() 

ia_results = json.loads(soup)

collected_dates = {}

homepages = []

for i in range(1, len(ia_results)):
    element = ia_results[i]
    date = element[1][:8]
    if date not in collected_dates:
        collected_dates[date] = ""
        homepages.append("http://web.archive.org/web/" + element[1] + "/" + element[2])
print (len(homepages))

output = codecs.open(website_name+"_homepages.txt", "w", "utf-8")

for element in homepages:
    output.write(element + "\n")
    
output.close()

In [ ]:
# the second part of this script extracts the links to the newspaper articles from each homepage and save them in a .txt
# it will take quite a bit

homepages = codecs.open(website_name+"_homepages.txt", "r", "utf-8").read().strip().split("\n")
articles_link = codecs.open(website_name+"_articles.txt", "w", "utf-8")

articles =  []

for i in range(len(homepages)):
    homepage = homepages[i]
    try:
        url_open = urllib.request.urlopen(homepage)
        soup = BeautifulSoup(url_open, "lxml")
        for link in soup.find_all('a', href=True):
            lk = link['href']
            true_url = "/".join(lk.split("/")[5:]).replace("https://","").replace("http://","")
            if website_name in lk and true_url not in set(articles):
                articles.append(true_url)
                if "http://web.archive.org" in lk:
                    articles_link.write(lk+"\n")
                else:
                    articles_link.write( "http://web.archive.org"+lk+"\n")

    
        print (i+1, " homepages processed over ", len(homepages), ", ",len(articles)," articles collected")
        
    except Exception as e:
        print (e)
        time.sleep(1)
        pass
            

output.close()

In [ ]:
'''
last part - for each article, it goes to the archived page and takes the content, then save it in a .tsv
bear in mind a couple of things: the date is the archival date, which could differ from the published date
if you want the published date you should scrape it from the content
same thing for the labeled-topic of the article
it will take a while
'''

articles = codecs.open(website_name+"_articles.txt", "r", "utf-8").read().strip().split("\n")

output = codecs.open(website_name+"_content-articles.tsv","w","utf-8")


for i in range(0,len(articles)):
    article_link = articles[i]
    try:
        url_open = urllib.request.urlopen(article_link)
                
        soup = BeautifulSoup(url_open, "lxml")
        title= soup.find('title')
        
        # lazy way of checking if you scraped a redirect page by mistake
        if len(title)>0:
            
            title= " ".join(soup.find('h1').text.split())
            
            date = article_link.split("/")[4][:8]
            
            text = ""
            
            #ugly step - i'll improve it 
            soup = str(soup).split("<!-- END WAYBACK TOOLBAR INSERT -->")[1]
            
            soup = BeautifulSoup(soup, "lxml")
           
            # simply take all the paragraphs
            for link in soup.find_all('p'):
                text+= link.text.replace("\n","").replace("\r","").replace("\t","") + " "
            
            # yea i know - i'm kinda lazy at the moment
            if len(text)>1:
                output.write(article_link +"\t"+date +"\t"+title+"\t"+ text+"\n")
            if (i % 10) == 0:
                print (i+1, " articles processed over ", len(articles))
                #time.sleep(1)
            
    except Exception as e:
        #print (e,i)
        #print (article_link)
        time.sleep(1)
        pass

output.close()